In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
from cad_rag.data.Dataset import CanadianCaseLawDocumentDatabase

In [ ]:
from pyarrow import parquet as pq
import pandas as pd
onca_path = os.path.join('/ubc/cs/research/nlp-raid/students/ethanz01/cs-masters/cad_rag/data/', 'canadian-case-law/ONCA/train.parquet')
print(onca_path)
onca = pd.read_parquet(os.path.join(onca_path, 'train.parquet'), engine='pyarrow')
# table = pq.read_table(onca_path)

/ubc/cs/research/nlp-raid/students/ethanz01/cs-masters/cad_rag/data/canadian-case-law/ONCA/train.parquet


In [ ]:
dataset = CanadianCaseLawDocumentDatabase()
print(dataset.get_dataset_names())
onca_dataset = dataset.get_dataset_by_name("ONCA")
# print(onca_dataset.iloc[0])


['CHRT', 'CMAC', 'FC', 'FCA', 'ONCA', 'RAD', 'RLLR', 'RPD', 'SCC', 'SST', 'TCC']


AttributeError: 'generator' object has no attribute 'iloc'

In [6]:
import multiprocessing as mp

In [5]:
data = CanadianCaseLawDocumentDatabase()
names = data.get_dataset_names()
print(names)
text_key = "unofficial_text_en"  # key for the main text field in the datasets
date_key = "document_date_en"  # key for the date field in the datasets

['CHRT', 'CMAC', 'FC', 'FCA', 'ONCA', 'RAD', 'RLLR', 'RPD', 'SCC', 'SST', 'TCC']


In [7]:
cols = ['dataset', 'citation_en', 'citation2_en', 'name_en', 'document_date_en',
       'url_en', 'scraped_timestamp_en', 'unofficial_text_en', 'citation_fr',
       'citation2_fr', 'name_fr', 'document_date_fr', 'url_fr',
       'scraped_timestamp_fr', 'unofficial_text_fr', 'upstream_license']

In [8]:
def write_samples(name, samples):
    for i in range(25):
        text = samples.iloc[i].loc[text_key]
        
        with open(f'text_samples/{name}_sample_{i}.txt', 'w', encoding='utf-8') as f:
            if text is not None:
                f.write(text)

In [9]:
def get_dataset(**kwargs):
    if 'name' not in kwargs:
        raise ValueError("Missing 'data' argument")
    if 'min_year' not in kwargs:
        kwargs['min_year'] = 2000
        print(f"Using default min_year: {kwargs['min_year']}")
    df = data.get_dataset_by_name(kwargs['name'])
    df = df[df[date_key].dt.year >= kwargs['min_year']]
    
    return df

In [ ]:
def sample_data(name):
    cur_data = data.get_dataset_by_name(name=name)
    samples = cur_data.sample(25, random_state=423)
    write_samples(name, samples)

In [9]:
# get average document length in words
for name in names[9:]:
    cur_data = data.get_dataset_by_name(name=name)
    cur_data = cur_data[cur_data[text_key].notnull()]
    cur_data['doc_length'] = cur_data[text_key].apply(lambda x: len(x.split()))
    avg_length = cur_data['doc_length'].mean()
    print(f"Dataset: {name}, Average document length (words): {avg_length}")

Dataset: RAD, Average document length (words): 3674.2939421916058
Dataset: RLLR, Average document length (words): 2102.854368932039


In [ ]:
# get average document length in words
for name in names[6:]:
    cur_data = data.get_dataset_by_name(name=name)
    cur_data = cur_data[cur_data[text_key].notnull()]
    cur_data['doc_length'] = cur_data[text_key].apply(lambda x: len(x.split()))
    avg_length = cur_data['doc_length'].mean()
    print(f"Dataset: {name}, Average document length (words): {avg_length}")

Dataset: FC, Average document length (words): 3759.013411420205
Dataset: RPD, Average document length (words): 3464.3092584336455
Dataset: CMAC, Average document length (words): 4826.1164383561645


In [8]:
# get average document length in words
for name in names:
    cur_data = data.get_dataset_by_name(name=name)
    cur_data = cur_data[cur_data[text_key].notnull()]
    cur_data['doc_length'] = cur_data[text_key].apply(lambda x: len(x.split()))
    avg_length = cur_data['doc_length'].mean()
    print(f"Dataset: {name}, Average document length (words): {avg_length}")

Dataset: SCC, Average document length (words): 6631.015669911914
Dataset: CHRT, Average document length (words): 6325.3430451127815
Dataset: TCC, Average document length (words): 4188.543439716312
Dataset: ONCA, Average document length (words): 2603.389711521444
Dataset: SST, Average document length (words): 2191.382885056085
Dataset: FCA, Average document length (words): 2931.2915455746365


: 

In [7]:
name = 'SCC'
testdf = get_dataset(name=name)

samples = testdf.sample(5, random_state=423)
print(samples)

write_samples(name, samples)

Using default min_year: 2000
     dataset  citation_en       citation2_en  \
6665     SCC  2000 SCC 67  [2000] 2 SCR 1067   
6491     SCC  2001 SCC 33   [2001] 1 SCR 911   
6879     SCC  2003 SCC 34   [2003] 1 SCR 835   
8553     SCC   2010 SCC 5   [2010] 1 SCR 132   
8133     SCC  2008 SCC 38   [2008] 2 SCR 326   

                                                name_en  \
6665                      Whirlpool Corp. v. Camco Inc.   
6491                                 Mitchell v. M.N.R.   
6879     Trociuk v. British Columbia (Attorney General)   
8553             Fullowka v. Pinkerton's of Canada Ltd.   
8133  Charkaoui v. Canada (Citizenship and Immigration)   

              document_date_en  \
6665 2000-12-15 00:00:00+00:00   
6491 2001-05-24 00:00:00+00:00   
6879 2003-06-06 00:00:00+00:00   
8553 2010-02-18 00:00:00+00:00   
8133 2008-06-26 00:00:00+00:00   

                                                 url_en  \
6665  https://decisions.scc-csc.ca/scc-csc/scc-csc/e...   
6491

### Supreme Court of Canada (SCC)

In [8]:
del testdf

In [ ]:
name = 'SCC'
cur_data = data.get_dataset_by_name(name=name)

samples = cur_data.sample(5, random_state=423)
print(samples)

write_samples(name, samples)

In [19]:
cur_data.iloc[0].loc[text_key]

'Cape Breton Cold Storage Co. Ltd. v. Rowlings\nCollection\nSupreme Court Judgments\nDate\n1929-05-27\nReport\n[1929] SCR 505\nJudges\nDuff, Lyman Poore; Mignault, Pierre-Basile; Newcombe, Edmund Leslie; Lamont, John Henderson; Smith, Robert\nOn appeal from\nNova Scotia\nSubjects\nProfessional law\nDecision Content\nSupreme Court of Canada\nCape Breton Cold Storage Co. Ltd. v. Rowlings, [1929] S.C.R. 505\nDate: 1929-05-27\nCape Breton Cold Storage Company, Limited (Defendant) Appellant;\nand\nG. A. R. Rowlings (Plaintiff) Respondent.\n1929: May 1, 2; 1929: May 27.\nPresent: Duff, Mignault, Newcombe, Lamont and Smith JJ.\nON APPEAL FROM THE SUPREME COURT OF NOVA SCOTIA, EN BANC\nSolicitor—Company—Director of company acting as its solicitor—Claim for payment for legal services—Whether a “trustee” within s. 56 of the Trustee Act, R.S.N.S., 1923, c. 212.\nPlaintiff, who was a director and vice-president of defendant company, acted as its solicitor (although not formally appointed as such) 

In [ ]:
cur_data

### Canadian Human Rights Tribunal (CHRT)

In [9]:
name = 'CHRT'
cur_data = data.get_dataset_by_name(name=name)

samples = cur_data.sample(5, random_state=423)
print(samples)

write_samples(name, samples)

     dataset   citation_en citation2_en  \
583     CHRT   2004 CHRT 1                
939     CHRT  2020 CHRT 39                
1032    CHRT   2009 CHRT 7                
813     CHRT   2014 CHRT 9                
245     CHRT  2013 CHRT 18                

                                                name_en  \
583                   Coulter v. Purolator Courier Ltd.   
939             Farmer v. Halifax Employers Association   
1032  Zhou v. National Research Council and Chander ...   
813   Marsden v. Public Works and Government Service...   
245                 Seeley v. Canadian National Railway   

              document_date_en  \
583  2004-01-06 00:00:00+00:00   
939  2020-12-23 00:00:00+00:00   
1032 2009-02-20 00:00:00+00:00   
813  2014-03-07 00:00:00+00:00   
245  2013-07-18 00:00:00+00:00   

                                                 url_en  \
583   https://decisions.chrt-tcdp.gc.ca/chrt-tcdp/de...   
939   https://decisions.chrt-tcdp.gc.ca/chrt-tcdp/de...   
1032

### Tax Court of Canada (TCC)

In [ ]:
name = 'TCC'
cur_data = data.get_dataset_by_name(name=name)

samples = cur_data.sample(5, random_state=423)
print(samples)

write_samples(name, samples)
del cur_data

     dataset   citation_en citation2_en                          name_en  \
725      TCC  2005 TCC 685                          Chandan v. The Queen   
4837     TCC  2015 TCC 156                   Gutbucket Inc. v. The Queen   
6021     TCC   2006 TCC 62                          Boucher v. The Queen   
4235     TCC  2018 TCC 227               118682 Canada Ltd. v. The Queen   
6860     TCC  2010 TCC 255                           Hayter v. The Queen   

              document_date_en  \
725  2005-10-20 00:00:00+00:00   
4837 2015-06-22 00:00:00+00:00   
6021 2005-12-06 00:00:00+00:00   
4235 2018-11-14 00:00:00+00:00   
6860 2010-05-07 00:00:00+00:00   

                                                 url_en  \
725   https://decision.tcc-cci.gc.ca/tcc-cci/decisio...   
4837  https://decision.tcc-cci.gc.ca/tcc-cci/decisio...   
6021  https://decision.tcc-cci.gc.ca/tcc-cci/decisio...   
4235  https://decision.tcc-cci.gc.ca/tcc-cci/decisio...   
6860  https://decision.tcc-cci.gc.ca/tcc-cc

### Ontario Court of Appeal (ONCA)

In [13]:
name = 'ONCA'
cur_data = data.get_dataset_by_name_as_df(name=name)

samples = cur_data.sample(25, random_state=423)
print(samples)

write_samples(name, samples)
del cur_data

      dataset    citation_en citation2_en  \
10550    ONCA   2010 ONCA 90                
16295    ONCA  2023 ONCA 863                
3807     ONCA  2022 ONCA 409                
860      ONCA  2009 ONCA 325                
10696    ONCA  2011 ONCA 341                
3402     ONCA  2017 ONCA 370                
3971     ONCA  2009 ONCA 815                
2438     ONCA   2017 ONCA 72                
14738    ONCA  2025 ONCA 236                
13503    ONCA   2025 ONCA 53                
7021     ONCA  2007 ONCA 748                
12715    ONCA  2015 ONCA 829                
6223     ONCA  2007 ONCA 697                
7710     ONCA  2019 ONCA 321                
7357     ONCA  2008 ONCA 101                
4961     ONCA  2013 ONCA 230                
597      ONCA  2009 ONCA 633                
8387     ONCA  2008 ONCA 653                
13413    ONCA   2025 ONCA 82                
12509    ONCA  2015 ONCA 766                
9032     ONCA  2021 ONCA 363                
9075     O

### Social Security Tribunal of Canada (SST)

In [6]:
name = 'SST'
cur_data = data.get_dataset_by_name(name=name)

samples = cur_data.sample(5, random_state=423)
print(samples)

write_samples(name, samples)

      dataset       citation_en citation2_en  \
802       SST   2016 SSTGDIS 85                
3789      SST    2015 SSTAD 648                
1310      SST  2017 SSTADEI 247                
7063      SST      2022 SST 842                
11436     SST     2024 SST 1729                

                                                 name_en  \
802    E. F. v. Minister of Employment and Social Dev...   
3789     A. A. v. Canada Employment Insurance Commission   
1310     Canada Employment Insurance Commission v. W. G.   
7063   CR v Minister of Employment and Social Develop...   
11436        DR v Canada Employment Insurance Commission   

               document_date_en  \
802   2017-10-19 00:00:00+00:00   
3789  2015-05-26 00:00:00+00:00   
1310  2017-06-27 00:00:00+00:00   
7063  2022-08-17 00:00:00+00:00   
11436 2024-11-20 00:00:00+00:00   

                                                  url_en  \
802    https://decisions.sst-tss.gc.ca/sst-tss/cpp-rp...   
3789   https://deci

### Federal Court of Appeal (FCA)

In [9]:
name = 'FCA'
cur_data = data.get_dataset_by_name(name=name)

samples = cur_data.sample(5, random_state=423)
print(samples)

write_samples(name, samples)

     dataset   citation_en citation2_en  \
351      FCA   2023 FCA 39                
276      FCA          None         None   
55       FCA   2013 FCA 16                
1386     FCA  2015 FCA 119                
3517     FCA   2003 FCA 28                

                                           name_en          document_date_en  \
351   Democracy Watch v. Canada (Attorney General) 2023-02-21 00:00:00+00:00   
276                                           None                       NaT   
55            Runchey v. Canada (Attorney General) 2013-01-24 00:00:00+00:00   
1386             Cole v. Canada (Attorney General) 2015-05-05 00:00:00+00:00   
3517         Saugeen First Nation #29 v. Sebastian 2003-01-22 00:00:00+00:00   

                                                 url_en  \
351   https://decisions.fca-caf.gc.ca/fca-caf/decisi...   
276                                                None   
55    https://decisions.fca-caf.gc.ca/fca-caf/decisi...   
1386  https://decisions.

### Federal Court (FC)

In [ ]:
name = 'FC'
# cur_data = data.get_dataset_by_name(name=name)

# samples = cur_data.sample(5, random_state=423)
# print(samples)

# write_samples(name, samples)

p = mp.Process(target=sample_data, args=(name,))
p.start()
p.join()

### Refugee Protection Division (IRB) (RPD)

In [19]:
name = 'RPD'
p = mp.Process(target=sample_data, args=(name,))
p.start()
p.join()

### Court Martial Appeal Court of Canada (CMAC)

In [20]:
name = 'CMAC'
p = mp.Process(target=sample_data, args=(name,))
p.start()
p.join()

### Refugee Appeal Division (IRB) (RAD)

In [21]:
name = 'RAD'
p = mp.Process(target=sample_data, args=(name,))
p.start()
p.join()

### Refugee Law Lab Reporter (RPD, IRB) (RLLR)

In [22]:
name = 'RLLR'
p = mp.Process(target=sample_data, args=(name,))
p.start()
p.join()